<a href="https://colab.research.google.com/github/PARAS759/Dog-Breed-identification/blob/main/Dog_Breed_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
import gc
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split

import tensorflow as tf
from tqdm.autonotebook import tqdm
from keras import Sequential
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten, Dense, BatchNormalization, Activation, Dropout
from keras.layers import Lambda, Input , GlobalAveragePooling2D , BatchNormalization
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
print("GPU", "available (YESS!!!)" if tf.config.list_physical_devices("GPU") else ("not available"))
tf.config.list_physical_devices("GPU")

In [ ]:
labels = pd.read_csv('/content/drive/MyDrive/Data/dog-breed-identification/labels.csv')
labels.head()

In [ ]:
labels.describe()

In [ ]:
# # function to show bar length
# def barw(ax):
#   for p in ax.patches:
#     val = p.get_width()
#     x = p.get_x()+ p.get_width()
#     y = p.get_y() + p.get_height()
#     ax.annotate(round(val,2),(x,y))

# # finding top dog brand
# plt.figure(figsize = (15,30))
# ax0 = sns.countplot(y = labels['breed'],order = labels['breed'].value_counts().index)
# barw(ax0)
# plt.show()

In [ ]:
# from IPython.display import display, Image
# Image('/content/drive/MyDrive/Data/dog-breed-identification/train/002211c81b498ef88e1b40b9abf84e1d.jpg')
# Image('/content/drive/MyDrive/Data/dog-breed-identification/train/0067dc3eab0b3c3ef0439477624d85d6.jpg')

In [ ]:
# if len(os.listdir('/content/drive/MyDrive/Data/dog-breed-identification/train'))==len(labels['id']):
#   print("Number of files and images are equal")
# else:
#   print('Not equal files and images')

In [ ]:
classes  = sorted(list(set(labels['breed'])))
n_classes = len(classes)
# print('Total unique breed {}'.format(n_classes))

# Mappint each label string to an intefer label
class_to_num = dict(zip(classes , range(n_classes)))
class_to_num

In [ ]:
input_shape = (250,250,3)

def images_to_array(directory , label_dataframe , target_size = input_shape):

  image_labels = label_dataframe['breed']
  images = np.zeros([len(label_dataframe) , target_size[0] , target_size[1] , target_size[2]],dtype = np.uint8)
  y = np.zeros([len(label_dataframe) , 1] , dtype = np.uint8)

  for ix,image_name in enumerate(tqdm(label_dataframe['id'].values)):
    img_dir = os.path.join(directory, image_name + '.jpg')
    img = load_img(img_dir,target_size = target_size)
    images[ix] = img
    del img
    dog_breed = image_labels[ix]
    y[ix] = class_to_num[dog_breed]
  y = to_categorical(y)
  return images,y


In [ ]:
import time 
t = time.time()

X,y = images_to_array('/content/drive/MyDrive/Data/dog-breed-identification/train',labels[:])
print('runtime in seconds : {}'.format(time.time()-t))

In [ ]:
# print(len(X))

In [ ]:
# n = 25
# # setup the figure
# plt.figure(figsize = (20,20))

# for i in range(n):
#   ax = plt.subplot(5,5,i+1)
#   plt.title(classes[np.where(y[i] == 1)[0][0]])
#   plt.imshow(X[i])

In [ ]:
lrr = ReduceLROnPlateau(monitor = 'val_acc',factor = .01 , patience = 3 , min_lr = 1e-5 , verbose = 1)
# prepare callbacks
EarlyStop = EarlyStopping(monitor = 'val_loss', patience = 10 , restore_best_weights = True)


In [ ]:
# Hyperparameters
batch_size = 64
epochs = 30
learn_rate = .001
sgd = SGD(learning_rate = learn_rate , momentum = .9 , nesterov = False)
adam = Adam(learning_rate = learn_rate , beta_1 = 0.9 , beta_2 = 0.999 , epsilon = None , amsgrad = False)


#Feature Extraction by help of pretrained models



In [ ]:
# function to extrract features from the dataset by a given pretrained mode
img_size = (250,250,3)

def get_features(mode_name , model_preprocessor , input_size , data):
  input_layer = Input(input_size)
  preprocessor = Lambda(model_preprocessor)(input_layer)
  base_model = mode_name(weights = 'imagenet' , include_top = False , input_shape = input_size)(preprocessor)
  avg  = GlobalAveragePooling2D()(base_model)
  feature_extractor = Model(inputs = input_layer , outputs = avg)

  # Extract feature
  feature_maps = feature_extractor.predict(data,verbose = 1)
  print('Feature maps shape : ',feature_maps.shape)
  return feature_maps

In [ ]:
# Extract features using InceptionV3
from keras.applications.inception_v3 import InceptionV3 , preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3 , inception_preprocessor , img_size , X )

In [ ]:
# Extract using Xception
from keras.applications.xception import Xception, preprocess_input
xception_preprocessor =  preprocess_input
xception_features = get_features(Xception , xception_preprocessor , img_size , X)


In [ ]:
# Extract features using InceptionResNetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2 , preprocess_input
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_features(InceptionResNetV2 , inc_resnet_preprocessor , img_size , X)

In [ ]:
# Extract features using NASNetLarge
from keras.applications.nasnet import NASNetLarge , preprocess_input
nasnet_preprocessor = preprocess_input
nasnet_features = get_features(NASNetLarge , nasnet_preprocessor,img_size , X)

In [ ]:
del X 
gc.collect()


#Combination of features

In [ ]:
# creating final featuremap by combining all extracted features
final_features  = np.concatenate([inception_features,xception_features,nasnet_features,inc_resnet_features,],axis = -1)
print('Final feature maps shape' , final_features.shape)

#Preparing Input Layer with 0.7 Dropout

In [ ]:
model = Sequential()
model.add(Dropout(0,7,input_shape = (final_features.shape[1],)))
model.add(Dense(n_classes,activation = 'softmax'))

model.compile(optimizer = adam,loss = 'categorical_crossentropy',metrics = ['accuracy'])

# Training the model...............
history = model.fit(final_features,y,batch_size = batch_size , epochs = epochs , validation_split = 0.2 , callbacks = [lrr,EarlyStop])


In [ ]:
# deleting to free up ram memory

del inception_features
del xception_features
del nasnet_features
del inc_resnet_features
del final_features
gc.collect()

In [ ]:
# Fuction to read images from test directory 
from __future__ import annotations
def images_to_array_test(test_path, img_size = (250,250,3)):
  test_filenames = [test_path+fname for fname in os.listdir(test_path)]

  data_size = len(test_filenames)
  images = np.zeros([data_size , img_size[0] , img_size[1],3],dtype = np.uint8)

  for ix,img_dir in enumerate(tqdm(test_filenames)):
    img = load_img(img_dir, target_size = img_size)
    images[ix] = img
    del img
  print('Output Datasize : ',images.shape)
  return images
test_data = images_to_array_test('/content/drive/MyDrive/Data/dog-breed-identification/test/',)

In [ ]:
# Extract features from test dataset
def extract_features(data):
  inception_features = get_features(InceptionV3 , inception_preprocessor , img_size , data)
  xception_features = get_features(Xception,xception_preprocessor , img_size , data)
  nasnet_features = get_features(NASNetLarge,nasnet_preprocessor , img_size , data)
  inc_resnet_features = get_features(InceptionResNetV2 , inc_resnet_preprocessor , img_size , data)

  final_features = np.concatenate([inception_features , xception_features , nasnet_features , inc_resnet_features],axis = -1)

  print('Final feature maps shape' , final_features.shape)

  return final_features

test_features = extract_features(test_data)


In [ ]:
# Predict test labels given test data features
pred  = model.predict(test_features)


In [ ]:
# First prediction
print(pred[0])
print(f"Max value (probability of prediction): {np.max(pred[0])}")
print(f"Sum: {np.sum(pred[0])}")
print(f"Max index: {np.argmax(pred[0])}")
print(f"Predicted Label: {classes[np.argmax(pred[0])]}")